In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pylab as plt

import seaborn as sns

from skspatial.objects import Line, Plane
from skspatial.plotting import plot_3d

from skspatial.objects import Line, Cylinder, Point, Points
from skspatial.plotting import plot_3d

import phasespace

import tensorflow

import bisect

import seaborn as sns

import dm_generation_tools as dgt
import detector_simulation_tools as dst

import time

####################################
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")


import pickle

In [ ]:
def generate_origins_and_directions(nevents=100, width=10):
    x = (width/2) - width*np.random.random(nevents)
    y = (width/2) - width*np.random.random(nevents)
    z = -10*np.ones(nevents)
    origins = np.array([x,y,z]).T
    
    # Generate betwen 0 and 1
    theta = np.arccos(np.random.random(nevents))
    phi = 2*np.pi*np.random.random(nevents)
    r = 100*np.ones(nevents)
    
    px = r*np.sin(theta)*np.cos(phi)
    py = r*np.sin(theta)*np.sin(phi)
    pz = r*np.cos(theta)
    
    directions = np.array([px, py, pz]).T

    return origins, directions


nevents = 100000
width = 1

origins, directions = generate_origins_and_directions(nevents=nevents, width=width)
pts0,pts1 = dst.intersect_finite_cylinder_x_np(origins=origins, directions=directions)

#print(pts0)
#print(pts1)

In [ ]:
fig, ax, detector = dst.draw_detector()

colors = ['k', 'r', 'b', 'g', 'm']

npoints = 0
#for idx,n in enumerate(range(len(pts0))):
for idx,n in enumerate(range(100)):

    #print(n)
    #print(pts0[n])
    #print(pts1[n])

    color = colors[n%len(colors)]
    #print(Pa1[n], Pb1[n])
    # Is it not a nan
    if pts0[n][0] == pts0[n][0]:
        pa,pb,line = dst.draw_points(pts0[n], pts1[n], ax=ax, color=color)
        if pa is not None or pb is not None:
            #print(pts0[n], pts1[n])
            npoints += 1
        if pa[2] > pb[2]:
            print(pa, pb)

ax.set_xlim(-15, 15)
ax.set_ylim(-10, 10)
ax.set_zlim(-10, 10)

ax.set_xlabel("X axis")
ax.set_ylabel("Y axis")
ax.set_zlabel("Z axis")

ax.set_title(f'Muons from width {width} m')

plt.tight_layout()

print(npoints)
plt.savefig(f'intersection_chords_muons_from_width_{width}.png')

In [ ]:
idx = ~np.isnan(pts0.T[0])

print(len(idx), len(idx[idx]))

px,py,pz = directions[idx].T
x0,y0,z0 = origins.T
x,y,z = origins[idx].T
pmag = np.sqrt(px**2 + py**2 + pz**2)

theta = np.arccos(pz/pmag)
phi = np.arctan2(py, px)

fig = plt.figure(figsize=(12,8))
fig.suptitle(f'muons from width {width} m')

plt.subplot(2,2,1)
plt.hist(np.cos(theta), bins=50)
plt.xlabel(r'Angle of entry $\cos\theta$', fontsize=14)

plt.subplot(2,2,2)
plt.hist(phi, bins=50)
plt.xlabel(r'Angle of entry $\phi$ (radians)', fontsize=14)

#'''
plt.subplot(2,3,4)
plt.plot(x0,y0, '.', markersize=0.5, label='Origin of muons - generated')
plt.xlabel(r'x (meters)', fontsize=12)
plt.ylabel(r'y (meters)', fontsize=12)
plt.legend()

plt.subplot(2,3,5)
plt.plot(x,y, '.', markersize=0.5, label='Origin of muons - that strike the detector')
plt.xlabel(r'x (meters)', fontsize=12)
plt.ylabel(r'y (meters)', fontsize=12)
plt.legend()

plt.subplot(2,3,6)
plt.hist(z, bins=50, label='Origin of muons')
plt.xlabel(r'z (meters)', fontsize=12)
plt.legend()
#'''

plt.tight_layout()

filename = f"detector_simulation_angle_of_entry_and_origin_from_width_{width}_meters.png"
plt.savefig(filename)
;

In [ ]:
# Do many

nevents = 100000
width = 100

plt.figure(figsize=(12,12))


for idxplot, width in enumerate([1, 5, 10, 25, 50, 75, 100]):

    print(idx, width)
    
    origins, directions = generate_origins_and_directions(nevents=nevents, width=width)
    pts0,pts1 = dst.intersect_finite_cylinder_x_np(origins=origins, directions=directions)


    idx = ~np.isnan(pts0.T[0])

    print(len(idx), len(idx[idx]))
    
    px,py,pz = directions[idx].T
    pmag = np.sqrt(px**2 + py**2 + pz**2)
    
    theta = np.arccos(pz/pmag)
    phi = np.arctan2(py, px)
    
    plt.subplot(4,2,1+idxplot)
    plt.hist(np.cos(theta), bins=50, range=(0,1), label=f'field width={width} m')
    plt.xlabel(r'Angle of entry $\cos\theta$', fontsize=14)

    plt.legend()
    
plt.tight_layout()
    
filename = "detector_simulation_angle_of_entry_for_different_sized_origin_planes.png"
plt.savefig(filename)
